# Imports

In [24]:
%pip install pandas numpy matplotlib seaborn scikit-learn xgboost
%pip freeze > requirements.txt
# pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [26]:
df = pd.read_csv('Dados/desafio_indicium_imdb.csv', index_col=0)

In [27]:
df.rename(columns={'Series_Title': 'title'}, inplace=True)

In [28]:
df_top_movies = pd.read_csv('Dados/top_year_movies.csv')
df = pd.merge(df, df_top_movies, on='title', how='left')
df = df.drop(columns=['win'])

# EDA

## Geral

In [29]:
df.shape

(1104, 36)

In [31]:
df.head(2)

,title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,id,link,year,duration,rating_mpa,rating_imdb,vote,budget,gross_world_wide,gross_us_canada,gross_opening_weekend,director,writer,star,genre,country_origin,filming_location,production_company,language,nomination,oscar
0,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch transfers control of his clandestine empire to his reluctant son.,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411",tt0068646,https://www.imdb.com/title/tt0068646,1972.0,2h 55m,R,9.2,21000000.0,6000000.0,2.503422e+08,136381073.0,302393.0,Francis Ford Coppola,"Mario Puzo, Francis Ford Coppola","Marlon Brando, Al Pacino, James Caan","Epic, Gangster, Tragedy, Crime, Drama",United States,"Forza dAgr, Messina, Sicily, Italy","Paramount Pictures, Albert S Ruddy Productions, Alfran Productions","English, Italian, Latin",31.0,0.0
1,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,"When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice.",84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444",tt0468569,https://www.imdb.com/title/tt0468569,2008.0,2h 32m,PG-13,9.0,3000000.0,185000000.0,1.009054e+09,534987076.0,158411483.0,Christopher Nolan,"Jonathan Nolan, Christopher Nolan, David S Goyer","Christian Bale, Heath Ledger, Aaron Eckhart","Action Epic, Epic, Superhero, Action, Crime, Drama, Thriller","United States, United Kingdom","Chicago, Illinois, USA","Warner Bros, Legendary Entertainment, Syncopy","English, Mandarin",164.0,0.0


In [32]:
df.describe()

,IMDB_Rating,Meta_score,No_of_Votes,year,rating_imdb,vote,budget,gross_world_wide,gross_us_canada,gross_opening_weekend,nomination,oscar
count,1104.000000,933.000000,1.104000e+03,783.000000,783.000000,7.830000e+02,6.490000e+02,6.900000e+02,6.520000e+02,5.930000e+02,783.000000,783.000000
mean,7.944203,77.993569,2.747243e+05,1996.899106,7.634100,1.423339e+06,5.165698e+07,1.839053e+08,8.497509e+07,1.763819e+07,38.754789,0.664112
std,0.271864,12.337613,3.172512e+05,16.551285,0.800431,4.014217e+06,1.310853e+08,3.214917e+08,1.195147e+08,3.622051e+07,54.019520,1.377877
min,7.600000,28.000000,2.508800e+04,1960.000000,2.500000,1.400000e+01,2.000000e+03,3.570000e+02,3.622000e+03,4.320000e+02,0.000000,0.000000
25%,7.700000,70.000000,5.669625e+04,1985.000000,7.600000,6.800000e+04,6.000000e+06,9.966327e+06,6.982804e+06,1.621790e+05,5.000000,0.000000
50%,7.900000,79.000000,1.432500e+05,2000.000000,7.800000,2.140000e+05,1.800000e+07,6.027637e+07,4.095555e+07,1.645502e+06,19.000000,0.000000
75%,8.100000,87.000000,3.793138e+05,2010.500000,8.000000,5.445000e+05,5.200000e+07,2.106674e+08,1.101160e+08,1.982869e+07,48.500000,1.000000
max,9.200000,100.000000,2.303232e+06,2024.000000,9.200000,2.600000e+07,2.500000e+09,2.923706e+09,9.366622e+08,3.571150e+08,382.000000,11.000000


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1104 entries, 0 to 1103
Data columns (total 36 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   title                  1104 non-null   object 
 1   Released_Year          1104 non-null   object 
 2   Certificate            999 non-null    object 
 3   Runtime                1104 non-null   object 
 4   Genre                  1104 non-null   object 
 5   IMDB_Rating            1104 non-null   float64
 6   Overview               1104 non-null   object 
 7   Meta_score             933 non-null    float64
 8   Director               1104 non-null   object 
 9   Star1                  1104 non-null   object 
 10  Star2                  1104 non-null   object 
 11  Star3                  1104 non-null   object 
 12  Star4                  1104 non-null   object 
 13  No_of_Votes            1104 non-null   int64  
 14  Gross                  925 non-null    object 
 15  id  

## Hipoteses

1️⃣ Relação entre IMDB_Rating e outras variáveis numéricas

Hipótese: Filmes com mais votos (No_of_Votes) tendem a ter nota mais alta ou mais estável.Visualização: Scatter plot (sns.scatterplot) ou correlação (df.corr()).

Hipótese: Filmes com maior Meta_score tendem a ter nota maior no IMDB.Visualização: Scatter plot e regressão (sns.regplot).

Hipótese: Filmes mais longos (Runtime) podem ter notas maiores.Visualização: Boxplot ou scatter plot.

2️⃣ Relação entre IMDB_Rating e variáveis categóricas

Hipótese: Alguns gêneros (Genre) tendem a receber notas mais altas.Visualização: Boxplot do IMDB_Rating por gênero (sns.boxplot(x='Genre', y='IMDB_Rating', data=df)).

Hipótese: Certificações etárias (Certificate) podem influenciar a nota (ex: filmes adultos vs familiares).Visualização: Boxplot ou violin plot.

Hipótese: Diretores ou atores populares (Director, Star1) podem impactar a nota.Visualização: Agrupar e calcular média de IMDB_Rating por diretor ou ator; gráfico de barras para os top 10.

3️⃣ Faturamento (Gross)

Hipótese: Filmes com maior faturamento tendem a ter maior nota?Visualização: Scatter plot IMDB_Rating vs Gross (lembrando de converter para numérico).

Hipótese: Filmes de certos gêneros têm maior faturamento.Visualização: Boxplot de Gross por gênero.

4️⃣ Texto do Overview

Hipótese: Palavras-chave no Overview podem indicar gênero ou impacto na nota.Visualização: Wordclouds por gênero ou correlação de frequência de palavras com IMDB_Rating.

5️⃣ Tempo de lançamento

Hipótese: Filmes mais recentes tendem a ter notas diferentes.Visualização: Scatter plot ou lineplot de IMDB_Rating por Released_Year.

# Modelo

In [34]:
target = 'IMDB_Rating'

In [35]:
X = df.drop(columns=[target, 'rating_imdb'])
y = df[target]

from sklearn.preprocessing import LabelEncoder

X_encoded = X.copy()

# Transformar colunas object em números
for col in X_encoded.select_dtypes(include=['object']).columns:
    X_encoded[col] = LabelEncoder().fit_transform(X_encoded[col].astype(str))
X = X_encoded.copy()

# Dividir treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Criar o modelo XGBoost Regressor
model = xgb.XGBRegressor(
    n_estimators=100,   # número de árvores
    learning_rate=0.1,  # taxa de aprendizado
    max_depth=5,        # profundidade máxima da árvore
    random_state=42
)

# Treinar o modelo
model.fit(X_train, y_train)

# Fazer previsões
y_pred = model.predict(X_test)

# Avaliar o modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R² Score: {r2:.2f}")

importances = model.feature_importances_

# Criar um DataFrame para organizar
feat_importances = pd.DataFrame({
    "Feature": X.columns,
    "Importance": importances
}).sort_values(by="Importance", ascending=False)

print(feat_importances)

Mean Squared Error: 0.04
R² Score: 0.41
                  Feature  Importance
12            No_of_Votes    0.194865
6              Meta_score    0.093385
17               duration    0.048832
28         country_origin    0.048674
19                   vote    0.045553
31               language    0.038649
16                   year    0.035742
20                 budget    0.034353
1           Released_Year    0.033829
23  gross_opening_weekend    0.030552
8                   Star1    0.030389
10                  Star3    0.027469
32             nomination    0.025488
21       gross_world_wide    0.023585
9                   Star2    0.021829
22        gross_us_canada    0.020922
5                Overview    0.019929
29       filming_location    0.019851
3                 Runtime    0.019752
4                   Genre    0.018060
25                 writer    0.017729
7                Director    0.016965
15                   link    0.016080
33                  oscar    0.015632
24        